In [25]:
import os
import pickle
import math
import copy
import nltk
from nltk.tokenize import word_tokenize
import csv

In [26]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/avijitmandal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
PATH = os.getcwd()
PATH_DATA = PATH + '/data'
PATH_DATA_en_BDNEWS24 = PATH + '/data/en_BDNEWS24'

In [28]:
dbfile = open('model_queries_11.pth', 'rb')     
newdict = pickle.load(dbfile)
dbfile.close

<function BufferedReader.close>

In [30]:
total_num = len(newdict.keys())

In [31]:
name_to_num = {}
num_to_name = {}
index = 0
for x in newdict.keys():
    name_to_num[x] = index
    num_to_name[index] = x
    index += 1

In [32]:
num_document = 0
document_vectors = {}
os.chdir(PATH_DATA_en_BDNEWS24)

for dirname in os.listdir(os.getcwd()):
    for filename in os.listdir(os.path.join(os.getcwd(),dirname)):
        num_document += 1
        document_vectors[filename] = {}

In [33]:
for key in newdict.keys():
    pos = name_to_num[key]
    for temp in newdict[key]:
        try:
            document_vectors[temp[0]][pos] = temp[1]
        except KeyError:
            print(keyError)

In [35]:
os.chdir(PATH_DATA)
query_file=open('queries_13.txt')
lines = query_file.readlines()

query_vectors = {}
queries = {}
queries[''] = 'Average'
for line in lines:
    
    line_list = line.split(',');
    query_id = line_list[0]
    queries[query_id] = line_list[1]
    words = word_tokenize(line_list[1])
    query_vectors[query_id] = {}
    
    for key in words:
        if key in name_to_num:
            pos = name_to_num[key]
            query_vectors[query_id][pos]=1    

In [36]:
def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1],reverse=True)
    return sub_li

# lnc.ltc Scheme

In [37]:
document_vectors1 = copy.deepcopy(document_vectors)
query_vectors1 = copy.deepcopy(query_vectors)

for x in document_vectors1:
    for y in document_vectors1[x]:
        document_vectors1[x][y]= 1+math.log(document_vectors1[x][y],10)  #(1 +log(tf))*1  --> ln

        
for x in query_vectors:
    for y in query_vectors[x]:
        #(1+log(tf))*log(N/df) -->lc
        query_vectors1[x][y]=(1+math.log(query_vectors1[x][y],10))*math.log(num_document/len(newdict[num_to_name[y]]),10)
        

In [38]:
priority_list=[]
ranked_list_A={}
for x in query_vectors1:
    query_norm=0
    for y in query_vectors1[x]:
        query_norm+=query_vectors1[x][y]*query_vectors1[x][y]
    query_norm=math.sqrt(query_norm) 
    
    for dv in document_vectors1:
        val=0
        for pos in query_vectors1[x]:
            if pos in document_vectors1[dv]:
                val+=query_vectors1[x][pos]*document_vectors1[dv][pos]
                
        doc_norm=0
        for y in document_vectors1[dv]:
            doc_norm+=document_vectors1[dv][y]*document_vectors1[dv][y]
        doc_norm=math.sqrt(doc_norm)
        
        if doc_norm!=0 and query_norm!=0:
            priority_list.append([dv,val/(doc_norm*query_norm)])
            
    Sort(priority_list)
    ranked_list_A[x]=priority_list[:50]
    priority_list.clear()

In [39]:
#Lnc.Lpc
document_vectors2 = copy.deepcopy(document_vectors)
query_vectors2 = copy.deepcopy(query_vectors)

for x in document_vectors2:
    total_freq = 0
    count = 0
    for y in document_vectors2[x]:
        count += 1
        total_freq += document_vectors2[x][y]
    for y in document_vectors2[x]:
        #(1 +log(tf))/(1+log(avg(tf)))  --> Ln
        document_vectors2[x][y]=(1+math.log(document_vectors2[x][y],10))/(1+math.log(total_freq/count,10))  

        
for x in query_vectors2:
    total_freq = 0
    count = 0
    for y in query_vectors2[x]:
        count += 1
        total_freq += query_vectors2[x][y]
    for y in query_vectors2[x]:
        #(1+log(tf))*log(N/df) -->Lp
        query_vectors2[x][y]=((1+math.log(query_vectors2[x][y],10))/(1+math.log(total_freq/count,10)))*max(0,math.log(num_document-len(newdict[num_to_name[y]])/(len(newdict[num_to_name[y]]))))
        


In [40]:
priority_list = []
ranked_list_B = {}
for x in query_vectors2:
    query_norm = 0
    for y in query_vectors2[x]:
        query_norm += query_vectors2[x][y]*query_vectors2[x][y]
    query_norm = math.sqrt(query_norm) 
    
    for dv in document_vectors2:
        val=0
        for pos in query_vectors2[x]:
            if pos in document_vectors2[dv]:
                val += query_vectors2[x][pos] * document_vectors2[dv][pos]
        doc_norm = 0
        for y in document_vectors2[dv]:
            doc_norm += document_vectors2[dv][y] * document_vectors2[dv][y]
        doc_norm = math.sqrt(doc_norm)
        #print(val/(doc_norm*query_norm))
        if doc_norm != 0 and query_norm != 0:
            priority_list.append([dv,val/(doc_norm*query_norm)])
    Sort(priority_list)
    ranked_list_B[x]=priority_list[:50]
    priority_list.clear()

In [41]:
#anc.apc
document_vectors3 = copy.deepcopy(document_vectors)
query_vectors3 = copy.deepcopy(query_vectors)

for x in document_vectors3:
    max_freq=0
    for y in document_vectors3[x]:
        max_freq=max(max_freq, document_vectors3[x][y])
        
    for y in document_vectors3[x]:
        #(0.5+(0.5*tf)/maximum(tf))  --> an
        document_vectors3[x][y]=(0.5+(0.5*document_vectors3[x][y])/max_freq)  

        
for x in query_vectors3:
    max_freq=0
    for y in query_vectors3[x]:
        max_freq=max(max_freq,query_vectors3[x][y])
        
    for y in query_vectors3[x]:
        #(0.5+(0.5*tf)/maximum(tf))*max(0,log((N-df)/df))  --> ap
        query_vectors3[x][y]=(0.5+(0.5*query_vectors3[x][y])/max_freq)*max(0,math.log(num_document-len(newdict[num_to_name[y]])/(len(newdict[num_to_name[y]]))))

In [42]:
priority_list = []
ranked_list_C = {}
for x in query_vectors3:
    #print(queryVectors3[x])
    query_norm = 0
    for y in query_vectors3[x]:
        query_norm += query_vectors3[x][y]*query_vectors3[x][y]
    query_norm = math.sqrt(query_norm) 
    
    for dv in document_vectors3:
        val=0
        for pos in query_vectors3[x]:
            if pos in document_vectors3[dv]:
                val+=query_vectors3[x][pos]*document_vectors3[dv][pos]
            else:
                val+=query_vectors3[x][pos]*0.5
        count=0        
        doc_norm=0
        for y in document_vectors3[dv]:
            doc_norm+=document_vectors3[dv][y]*document_vectors3[dv][y]
            count+=1
        doc_norm=math.sqrt(doc_norm+(total_num-count)*0.25)
        #print(val/(doc_norm*query_norm))
        if doc_norm!=0 and query_norm!=0:
            priority_list.append([dv,val/(doc_norm*query_norm)])
    Sort(priority_list)
    ranked_list_C[x]=priority_list[:50]
    priority_list.clear()

In [43]:
os.chdir(PATH)
csv_file = open("PAT2_13_ranked_list_A.csv", "w")
writer = csv.writer(csv_file)
for key in ranked_list_A:
    for val in ranked_list_A[key]:
        writer.writerow([key, val[0]])
csv_file.close()

In [44]:
csv_file = open("PAT2_13_ranked_list_B.csv", "w")
writer = csv.writer(csv_file)
for key in ranked_list_B:
    for val in ranked_list_B[key]:
        writer.writerow([key, val[0]])
csv_file.close()

In [45]:
csv_file = open("PAT2_13_ranked_list_C.csv", "w")
writer = csv.writer(csv_file)
for key in ranked_list_C:
    for val in ranked_list_C[key]:
        writer.writerow([key, val[0]])
csv_file.close()

# part 2B

In [46]:

import pandas as pd

os.chdir(PATH_DATA)
file_name = 'rankedRelevantDocList.xlsx'
xl_file = pd.ExcelFile(file_name)

dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}['RelevantDocs']

In [47]:
relevant = {}
for index, row in dfs.iterrows():
    
    Query_ID, Document_ID, Relevance_Score, = str(row['Query_ID']), str(row['Document_ID']), int(row['Relevance_Score'])
    if Query_ID not in relevant:
        relevant[Query_ID] = {}
        
    relevant[Query_ID][Document_ID] = Relevance_Score

In [48]:
def evaluate(ranked_list, relevant, write_file = False, file_name=None):
    
    mAP_10, mAP_20 = 0.0, 0.0
    averNDCG_10, averNDCG_20 = 0.0, 0.0
    
    ap_10, ap_20, query_id = [], [], []
    query = []
    ndcg_10, ndcg_20 = [], []
    
    
    
    for key in ranked_list:
        
        query_id.append(key)
        
        if key not in relevant:
            print("key: ", key + ", Data Not provided!")
            
            ap_10.append(0)
            ap_20.append(0)
            ndcg_10.append(0)
            ndcg_20.append(0)
            
        else:

            count = 0
            score = 0.0
            relevant_count = 0

            dcg_i = 0.0

            Query_ID = key
            NDCG = []
            
            for doc, _ in ranked_list[Query_ID]:
                if doc in relevant[Query_ID]:
                    NDCG.append(relevant[Query_ID][doc])
                else:
                    NDCG.append(0)

            NDCG.sort()
            NDCG.reverse()
            
            for i in range(1, len(NDCG)):
                NDCG[i] = NDCG[i-1] + NDCG[i]/math.log(i+1, 10)
                
            for doc, _ in ranked_list[key]:
                count += 1
                if doc in relevant[key]:
                    relevant_count += 1
                    if count == 1:
                        dcg_i += relevant[Query_ID][doc] 
                    else:
                        dcg_i += (relevant[Query_ID][doc] / math.log(count, 10)) 
                score += (relevant_count / count)
                if count == 10:
                    print(key, "Average @10: ", score/10)
                    ndcg_val = NDCG[count-1]
                    if ndcg_val == 0:
                        ndcg_val = 1
                    print(key, "NDCG @10: ", dcg_i/ndcg_val)
                    
                    ap_10.append(score/10)
                    ndcg_10.append(dcg_i/ndcg_val)
                    
                    mAP_10 += score/10
                    averNDCG_10 += dcg_i/ndcg_val
                    

                if count == 20:
                    
                    ndcg_val = NDCG[count-1]
                    if ndcg_val == 0:
                        ndcg_val = 1
                    
                    print(key, "Average @20: ", score/20)
                    print(key, "NDCG @20: ", dcg_i/ndcg_val)
                    mAP_20 += score/20
                    averNDCG_20 += dcg_i/ndcg_val
                    ap_20.append(score/20)
                    ndcg_20.append(dcg_i/ndcg_val)
                    break
        print("____________________________________________________________________________________________")
    print(":::Average Values:::")
    m = len(ranked_list)
    print("mAP@10: ", mAP_10/m)
    print("mAP@20: ", mAP_20/m)
    print("averNDCG@10: ", averNDCG_10/m)
    print("averNDCG@20: ", averNDCG_20/m)
    
    query_id.append('')
    ap_10.append(mAP_10/m)
    ap_20.append(mAP_20/m)
    ndcg_10.append(averNDCG_10/m)
    ndcg_20.append(averNDCG_20/m)
        
        
    if not write_file:
        return
    
    with open(file_name, "w") as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(['query_id', 'query','AP@10', 'AP@20', 'NDCG@10', 'NDCG@20'])
        for i in range(0, len(query_id)):
            writer.writerow([query_id[i], queries[query_id[i]], ap_10[i],ap_20[i], ndcg_10[i], ndcg_20[i]])

In [49]:
print("Writting for Ranked List A")
os.chdir(PATH)
evaluate(ranked_list_A, relevant, True, 'PAT2_13_metrics_A.csv')

Writting for Ranked List A
126 Average @10:  0.5875396825396825
126 NDCG @10:  0.42150111552179875
126 Average @20:  0.6830447740683809
126 NDCG @20:  0.530448096191484
____________________________________________________________________________________________
127 Average @10:  0.398531746031746
127 NDCG @10:  0.6224331701116694
127 Average @20:  0.36645872380973
127 NDCG @20:  0.6224331701116694
____________________________________________________________________________________________
128 Average @10:  0.7382539682539683
128 NDCG @10:  0.8854241009314676
128 Average @20:  0.5028812647620697
128 NDCG @20:  0.8854241009314676
____________________________________________________________________________________________
129 Average @10:  0.7071031746031746
129 NDCG @10:  0.5566893415676994
129 Average @20:  0.8201130171428159
129 NDCG @20:  0.6461855178482026
____________________________________________________________________________________________
130 Average @10:  0.634920634920635


In [50]:
os.chdir(PATH)
print("Writting for Ranked List B")
evaluate(ranked_list_B, relevant, True, 'PAT2_13_metrics_B.csv')

Writting for Ranked List B
126 Average @10:  0.5542063492063491
126 NDCG @10:  0.4071306787743084
126 Average @20:  0.6382173150207207
126 NDCG @20:  0.5135777753806591
____________________________________________________________________________________________
127 Average @10:  0.2928968253968254
127 NDCG @10:  0.1558158733809604
127 Average @20:  0.1798869828571841
127 NDCG @20:  0.1558158733809604
____________________________________________________________________________________________
128 Average @10:  0.1928968253968254
128 NDCG @10:  3.321928094887363
128 Average @20:  0.1298869828571841
128 NDCG @20:  3.321928094887363
____________________________________________________________________________________________
129 Average @10:  0.49309523809523814
129 NDCG @10:  0.47213728466100985
129 Average @20:  0.6462319085713049
129 NDCG @20:  0.5867785674475284
____________________________________________________________________________________________
130 Average @10:  0.6449206349206

In [51]:
print("Writting for Ranked List C")
os.chdir(PATH)
evaluate(ranked_list_C, relevant, True, 'PAT2_13_metrics_C.csv')

Writting for Ranked List C
126 Average @10:  0.956388888888889
126 NDCG @10:  0.6032077779289651
126 Average @20:  0.8162898777778806
126 NDCG @20:  0.5978427765924025
____________________________________________________________________________________________
127 Average @10:  0.06456349206349207
127 NDCG @10:  1.285097208938469
127 Average @20:  0.06572031619051744
127 NDCG @20:  1.285097208938469
____________________________________________________________________________________________
128 Average @10:  0.0
128 NDCG @10:  0.0
128 Average @20:  0.0
128 NDCG @20:  0.0
____________________________________________________________________________________________
129 Average @10:  0.5142063492063491
129 NDCG @10:  0.617067936648268
129 Average @20:  0.6902260342856319
129 NDCG @20:  0.719515197178858
____________________________________________________________________________________________
130 Average @10:  0.44202380952380943
130 NDCG @10:  0.24580917778432226
130 Average @20:  0.372